# **TPS Jan, 2022**

## **Catboost Baseline**

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')

display(train, test)

# **Preprocessing**

In [6]:
all_data = pd.concat([train, test])
all_data

In [11]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['day'] = all_data['date'].dt.day
all_data['dayofweek'] = all_data['date'].dt.dayofweek
all_data['dayofmonth'] = all_data['date'].dt.days_in_month
all_data['dayofyear'] = all_data['date'].dt.dayofyear
all_data['weekday'] = all_data['date'].dt.weekday

In [14]:
all_data.drop(columns = ['num_sold', 'date', 'row_id'], inplace = True)
all_data

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in ['country', 'product', 'store']:
    all_data[col] = le.fit_transform(all_data[col])
    
    
all_data

In [16]:
train2 = all_data[:len(train)]
test2 = all_data[len(train):]
y = train['num_sold']

# **Modeling**

* Used TimeSeriesSplit

In [29]:
from sklearn.model_selection import TimeSeriesSplit
kfold = TimeSeriesSplit()

In [37]:
from catboost import CatBoostRegressor

test_pred = []
for train_id, test_id in kfold.split(train2):
    x_train, y_train = train2.iloc[train_id], y.iloc[train_id]
    x_valid, y_valid = train2.iloc[test_id], y.iloc[test_id]
    
    cat = CatBoostRegressor(n_estimators = 10000)
    cat.fit(x_train, y_train, eval_set = (x_valid, y_valid), verbose = 1000, early_stopping_rounds = 1000)
    test_pred.append(cat.predict(test2))

In [46]:
sold = np.mean(test_pred, axis = 0)

In [47]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')
sub['num_sold'] = sold
sub.to_csv('sub.csv', index = False)